<a href="https://colab.research.google.com/github/konarksharma2001/Machine-Learning/blob/main/Naive_Bayes_Classifier.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [15]:
import pandas as pd

In [16]:
df = pd.read_csv('weather.csv')
df.head()

,Outlook,Temp,Humidity,Windy,Play
0,rainy,hot,high,0,0
1,rainy,hot,high,1,0
2,overcast,hot,high,0,1
3,sunny,mild,high,0,1
4,sunny,cool,normal,0,1


##Preprocessing

In [17]:
X_train = df.iloc[:,0:4]
X_train

,Outlook,Temp,Humidity,Windy
0,rainy,hot,high,0
1,rainy,hot,high,1
2,overcast,hot,high,0
3,sunny,mild,high,0
4,sunny,cool,normal,0
5,sunny,cool,normal,1
6,overcast,cool,normal,1
7,rainy,mild,high,0
8,rainy,cool,normal,0
9,sunny,mild,normal,0


In [18]:
y_train =  df.iloc[:,4]
y_train

0     0
1     0
2     1
3     1
4     1
5     0
6     1
7     0
8     1
9     1
10    1
11    1
12    1
13    0
Name: Play, dtype: int64

In [19]:
df_test = pd.read_csv('weather_test.csv')
df_test

,Outlook,Temp,Humidity,Windy,Play
0,rainy,cool,high,1,0
1,overcast,mild,normal,0,1


In [20]:
X_test = df_test.iloc[:,0:4]
X_test

,Outlook,Temp,Humidity,Windy
0,rainy,cool,high,1
1,overcast,mild,normal,0


In [21]:
y_test = df_test.iloc[:,4]
y_test

0    0
1    1
Name: Play, dtype: int64

###Prior Probability

In [13]:
import numpy as np
train_size=X_train.shape[0]
class_priors={}
for outcome in np.unique(y_train):
    outcome_count = sum(y_train == outcome)    
    class_priors[outcome] = outcome_count / train_size
print(class_priors)

{0: 0.35714285714285715, 1: 0.6428571428571429}


In [14]:
features=list(X_train.columns)
likelihoods={}
for outcome in np.unique(y_train):
    outcome_count = sum(y_train == outcome)
    for feature in features:
        for feat_value in np.unique(X_train[feature]):
            count=0
            for i in range(len(X_train)):
                if(X_train[feature][i]==feat_value and y_train[i]==outcome):
                    count=count+1
            likelihoods[(feature,feat_value,outcome)]=(count+1)/(outcome_count+(len(features)))

In [22]:
likelihoods

{('Humidity', 'high', 0): 0.5555555555555556,
 ('Humidity', 'high', 1): 0.3076923076923077,
 ('Humidity', 'normal', 0): 0.2222222222222222,
 ('Humidity', 'normal', 1): 0.5384615384615384,
 ('Outlook', 'overcast', 0): 0.1111111111111111,
 ('Outlook', 'overcast', 1): 0.38461538461538464,
 ('Outlook', 'rainy', 0): 0.4444444444444444,
 ('Outlook', 'rainy', 1): 0.23076923076923078,
 ('Outlook', 'sunny', 0): 0.3333333333333333,
 ('Outlook', 'sunny', 1): 0.3076923076923077,
 ('Temp', 'cool', 0): 0.2222222222222222,
 ('Temp', 'cool', 1): 0.3076923076923077,
 ('Temp', 'hot', 0): 0.3333333333333333,
 ('Temp', 'hot', 1): 0.23076923076923078,
 ('Temp', 'mild', 0): 0.3333333333333333,
 ('Temp', 'mild', 1): 0.38461538461538464,
 ('Windy', 0, 0): 0.3333333333333333,
 ('Windy', 0, 1): 0.5384615384615384,
 ('Windy', 1, 0): 0.4444444444444444,
 ('Windy', 1, 1): 0.3076923076923077}

##Prediction for Test Dataset

In [23]:
a = len(np.unique(y_train))
a

2

In [24]:
prob=np.ones((a,len(X_test)),dtype=np.float)
prob

array([[1., 1.],
       [1., 1.]])

In [25]:
for outcome in(np.unique(y_train)):
    outcome_count = sum(y_train == outcome)
    for feature in features:
        for i in range(len(X_test)):
            if  (feature,X_test[feature][i],outcome) in likelihoods.keys():
                prob[outcome][i]=prob[outcome][i]*likelihoods[(feature,X_test[feature][i],outcome)]
            else:
                prob[outcome][i]=prob[outcome][i]*(1/(outcome_count+len(features)))

###Multiplying probabilities with class prior probabilities

In [26]:
for i in range(prob.shape[0]):
    prob[i][:]=prob[i][:]*class_priors[i]

In [27]:
#Predict the labels

Y_label=np.zeros(len(y_test))
for i in range(len(X_test)):
    if (prob[1,i]>=prob[0,i]):
        Y_label[i]=1

In [28]:
#Performance Evaluation

from sklearn import metrics
print(metrics.classification_report(y_test,Y_label))
print(metrics.confusion_matrix(y_test,Y_label))

              precision    recall  f1-score   support

           0       1.00      1.00      1.00         1
           1       1.00      1.00      1.00         1

    accuracy                           1.00         2
   macro avg       1.00      1.00      1.00         2
weighted avg       1.00      1.00      1.00         2

[[1 0]
 [0 1]]
